In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Aniridia/GSE137997'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Conjunctival mRNA and miRNA expression profiles in congenital aniridia are genotype and phenotype dependent"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['age: 20', 'age: 28', 'age: 38', 'age: 57', 'age: 26', 'age: 18', 'age: 36', 'age: 42', 'age: 55', 'age: 54', 'age: 34', 'age: 51', 'age: 46', 'age: 52', 'age: 53', 'age: 40', 'age: 39', 'age: 59', 'age: 32', 'age: 37', 'age: 29', 'age: 19', 'age: 25', 'age: 22'], 1: ['gender: F', 'gender: M', 'gender: W'], 2: ['disease: AAK', 'disease: healthy control'], 3: ['Stage: Severe', 'Stage: Mild', 'Stage: NA'], 4: ['tissue: conjunctival cells']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 2
age_row = 0
gender_row = 1
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Define conversion functions
def convert_trait(value):
    trait_dict = {
        'AAK': 1,
        'healthy control': 0
    }
    result = value.split(': ')[1].strip()
    return trait_dict.get(result, None)

def convert_age(value):
    try:
        result = value.split(': ')[1].strip()
        return float(result)
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    gender_dict = {
        'F': 0,
        'M': 1,
        'W': None  # Treat 'W' as unknown gender
    }
    result = value.split(': ')[1].strip()
    return gender_dict.get(result, None)

# Save cohort information
save_cohort_info('GSE137997', './preprocessed/Aniridia/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and save clinical feature data
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Aniridia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Aniridia/trait_data/GSE137997.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4096389': [1.0, 20.0, 0.0], 'GSM4096390': [1.0, 20.0, 0.0], 'GSM4096391': [1.0, 28.0, 0.0], 'GSM4096392': [1.0, 20.0, 0.0], 'GSM4096393': [1.0, 38.0, 0.0], 'GSM4096394': [1.0, 57.0, 1.0], 'GSM4096395': [1.0, 26.0, 0.0], 'GSM4096396': [1.0, 18.0, 1.0], 'GSM4096397': [1.0, 36.0, 0.0], 'GSM4096398': [1.0, 42.0, 0.0], 'GSM4096399': [1.0, 18.0, 0.0], 'GSM4096400': [1.0, 42.0, 0.0], 'GSM4096401': [1.0, 36.0, 1.0], 'GSM4096402': [1.0, 28.0, 0.0], 'GSM4096403': [1.0, 55.0, 0.0], 'GSM4096404': [1.0, 54.0, 1.0], 'GSM4096405': [1.0, 34.0, 1.0], 'GSM4096406': [1.0, 51.0, 0.0], 'GSM4096407': [1.0, 46.0, 0.0], 'GSM4096408': [1.0, 52.0, 0.0], 'GSM4096409': [0.0, 53.0, nan], 'GSM4096410': [0.0, 54.0, 1.0], 'GSM4096411': [0.0, 40.0, nan], 'GSM4096412': [0.0, 55.0, nan], 'GSM4096413': [0.0, 57.0, nan], 'GSM4096414': [0.0, 28.0, nan], 'GSM4096415': [0.0, 39.0, nan], 'GSM4096416': [0.0, 59.0, nan], 'GSM4096417': [0.0, 20.0, nan], 'GSM4096418': [0.0, 32.0, 1.0], 'GSM4096419': [0.0, 37.0, 1.0], 'GSM409

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502',
       'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529',
       'A_19_P00315541', 'A_19_P00315543', 'A_19_P00315551', 'A_19_P00315581',
       'A_19_P00315584', 'A_19_P00315593', 'A_19_P00315603', 'A_19_P00315625',
       'A_19_P00315627', 'A_19_P00315631', 'A_19_P00315641', 'A_19_P00315647'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define the keys for the identifiers and gene symbols.
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes.
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Aniridia/gene_data/GSE137997.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Aniridia')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE137997', './preprocessed/Aniridia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Aniridia/GSE137997.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Aniridia', the least common label is '0.0' with 5 occurrences. This represents 20.00% of the dataset.
The distribution of the feature 'Aniridia' in this dataset is fine.

Quartiles for 'Age':
  25%: 26.0
  50% (Median): 36.0
  75%: 46.0
Min: 18.0
Max: 57.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 10 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Aniridia/cohort_info.json
